In [1]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"  

import numpy as np
from datasets import load_dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import f1_score, classification_report
import torch


/home/mahdi/Desktop/emotion/emo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())

True


In [3]:
from datasets import load_dataset

dataset = load_dataset("go_emotions")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})

In [5]:
dataset['train'].features['labels'].feature.names

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [6]:
n=59
label_names = dataset['train'].features['labels'].feature.names
label_indices = dataset['train'][n]['labels']
emotions = [label_names[i] for i in label_indices]
sample = dataset['train'][n]
print(sample)
emotions

{'text': "Thanks DB, I'll see if I can find the book", 'labels': [15], 'id': 'eejcgvu'}


['gratitude']

In [7]:
dataset['train'].to_pandas().head()

,text,labels,id
0,My favourite food is anything I didn't have to...,[27],eebbqej
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj


In [8]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(tokenize, batched=True)

In [9]:
def format_labels(example):
    labels = np.zeros(28)  # 27 emotions + neutral
    for label in example["labels"]:
        labels[label] = 1
    example["label"] = labels
    return example

tokenized_dataset = tokenized_dataset.map(format_labels)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [10]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=28,
    problem_type="multi_label_classification"
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = (logits > 0).astype(int)
    return {
        "f1_micro": f1_score(labels, preds, average="micro"),
        "f1_macro": f1_score(labels, preds, average="macro")
    }


In [12]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./roberta-goemotions",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_197583/2195567770.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro
1,0.091600,0.090242,0.529394,0.319070
2,0.083700,0.084003,0.569150,0.428242
3,0.075700,0.082950,0.579136,0.441348
4,0.066400,0.083795,0.584968,0.455659


TrainOutput(global_step=10856, training_loss=0.08588281226074687, metrics={'train_runtime': 4509.5066, 'train_samples_per_second': 38.505, 'train_steps_per_second': 2.407, 'total_flos': 4.569726889377792e+16, 'train_loss': 0.08588281226074687, 'epoch': 4.0})

In [14]:
results = trainer.evaluate(tokenized_dataset["test"])
print(results)

{'eval_loss': 0.08272188156843185, 'eval_f1_micro': 0.5897251661577151, 'eval_f1_macro': 0.45628601092730064, 'eval_runtime': 41.2407, 'eval_samples_per_second': 131.593, 'eval_steps_per_second': 8.244, 'epoch': 4.0}


In [19]:
def classify_text(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs.to('cuda'))
        probs = torch.sigmoid(outputs.logits).squeeze().cpu().numpy()
    threshold = 0.5
    predicted_labels = np.where(probs > threshold)[0]
    return predicted_labels, probs[predicted_labels]


In [33]:
test_text = "I am jealous"
labels, scores = classify_text(test_text)
print("\nPredicted emotion indices:", label_names[int(labels)])
print("Confidence scores:", scores)



Predicted emotion indices: annoyance
Confidence scores: [0.50292605]


/tmp/ipykernel_197583/1677690421.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("\nPredicted emotion indices:", label_names[int(labels)])
